In [1]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import re
import pandas_profiling as pp
import os
import pickle
import matplotlib.pyplot as plt
import zipfile
from pathlib import Path
from pandas.tseries.offsets import *

# Default plot configurations
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = (16,8)
plt.rcParams['figure.dpi'] = 150
sns.set()

In [2]:
policies = pd.read_csv("COVID-19 US State Policies database 8.14.2020.csv")

In [3]:
policies = policies.dropna(axis=0, how='all')
policies_definitions = policies.loc[0:3]
policies_definitions

,STATE,POSTCODE,STEMERG,CLSCHOOL,CLDAYCR,CLNURSHM,STAYHOME,END_STHM,CLBSNS,END_BSNS,...,PDSKLV,MEDEXP,POPDEN18,POP18,SQML,HMLS19,UNEMP18,POV18,RISKCOV,DEATH18
0,State,state,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,...,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line ...,Percent at risk for serious illness due to COVID,All-cause deaths 2018
1,category,postcode,emergency,school_closure,day_care_closure,nursing_home_visit_ban,shelter,shelter,business_closure,business_closure,...,paid_sick_leave,medicaid_expansion,population_density,population,area,homeless,unemployment,poverty,at_risk,all_cause_deaths
2,type,note,start,start,start,start,start,end,start,end,...,attribute,attribute,quantity,quantity,quantity,quantity,quantity,quantity,quantity,quantity
3,unit,text,date,date,date,date,date,date,date,date,...,flag,flag,people/sq mi,people,sq mi,people,percent,percent,percent,people/year


In [4]:
# policies

In [5]:
policies = policies.rename(columns=policies.iloc[0])
policies = policies.drop(axis=0, index=[0, 1, 2, 3])
policies = policies.reset_index().drop(axis=1, columns='index')


In [6]:
policies

,State,state,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,...,Paid sick leave,Medicaid Expansion,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2018
0,Alabama,AL,3/13/20,3/19/20,3/20/20,3/19/20,4/4/20,4/30/20,3/28/20,4/30/20,...,0,0,93.24,4887871,52420,"3,261",5.6,16.8,43.1,"54,352"
1,Alaska,AK,3/11/20,3/16/20,0,0,3/28/20,4/24/20,3/28/20,4/24/20,...,0,1,1.11,737438,665384,"1,907",6.8,10.9,32.8,"4,453"
2,Arizona,AZ,3/11/20,3/16/20,0,0,3/31/20,5/16/20,3/30/20,5/8/20,...,1,1,62.91,7171646,113990,"10,007",5.4,14,39.1,"59,282"
3,Arkansas,AR,3/11/20,3/17/20,0,3/13/20,0,0,0,5/4/20,...,0,1,56.67,3013825,53179,"2,717",4.5,17.2,43.5,"32,336"
4,California,CA,3/4/20,0,0,0,3/19/20,0,3/19/20,5/8/20,...,1,1,241.65,39557045,163695,"151,278",5.5,12.8,33.3,"268,818"
5,Colorado,CO,3/11/20,3/23/20,0,3/12/20,3/26/20,4/27/20,3/26/20,5/1/20,...,0,1,54.72,5695564,104094,"9,619",3.9,9.6,31.3,"38,526"
6,Connecticut,CT,3/10/20,3/17/20,0,3/9/20,0,0,3/23/20,5/20/20,...,1,1,644.54,3572665,5543,"3,033",5.5,10.4,36,"31,230"
7,Delaware,DE,3/13/20,3/16/20,4/6/20,0,3/24/20,6/1/20,3/24/20,5/8/20,...,0,1,388.58,967171,2489,921,5.7,12.5,41.3,"9,433"
8,District of Columbia,DC,3/11/20,3/16/20,0,0,4/1/20,5/29/20,3/25/20,5/29/20,...,1,1,"11,496.81","702,455",61.1,6521,7.5,16.2,31.8,"5,008"
9,Florida,FL,3/9/20,3/17/20,0,3/15/20,4/3/20,5/18/20,0,5/18/20,...,0,0,323.9,21299325,65758,"28,328",5.2,13.6,42.1,"205,426"


In [8]:
# important/relevant policies to small businesses
# 'Closed day cares', 'Date banned visitors to nursing homes', 'Stay at home/ shelter in place', 'End/relax stay at home/shelter in place', 
# 'Closed non-essential businesses', 'Began to reopen businesses', 'Mandate face mask use by all individuals in public spaces', 'Mandate face mask use by employees in public-facing businesses',
# 'Closed restaurants except take out', 'Reopen restaurants', 'Initially reopen restaurants for outdoor dining only', 'Closed gyms', 'Reopened gyms', 'Closed movie theaters', 'Reopened movie theaters',
# 'Stop Initiation of Evictions overall or due to COVID related issues', 'Stop enforcement of evictions overall or due to COVID related issues',
# 'Renter grace period or use of security deposit to pay rent', 'Order freezing utility shut offs', 'Froze mortgage payments',

# SEPARATE BY CATEGORIES OF POLICIES:
# Category: impacting overall
# 'Stay at home/ shelter in place', 'End/relax stay at home/shelter in place', 'Closed non-essential businesses', 'Began to reopen businesses'

# Category: particular small businesses
# 'Closed restaurants except take out', 'Reopen restaurants', 'Initially reopen restaurants for outdoor dining only', 'Closed gyms', 'Reopened gyms', 'Closed movie theaters', 'Reopened movie theaters'

# Category: not sure if applicable to small businesses
# 'Stop Initiation of Evictions overall or due to COVID related issues', 'Stop enforcement of evictions overall or due to COVID related issues', 'Renter grace period or use of security deposit to pay rent', 'Order freezing utility shut offs', 'Froze mortgage payments'
# 'Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined',
#        'Expand eligibility to high-risk individuals in preventative quarantine',
#        'Expand eligibility of unemployment insurance to those who have lost childcare/school closures',
#        'Extend the amount of time an individual can be on unemployment insurance',
#        'Weekly unemployment insurance maximum amount (dollars)',
#        'Weekly unemployment insurance maximum amount with extra stimulus (through July 21, 2020) (dollars)',
#        'Unemployment insurance maximum duration (weeks)',
#        'Unemployment insurance maximum duration with Pandemic Emergency Unemployment Compensation CARES extension (weeks)'



In [10]:
rel_policies = policies[['State', 'Stay at home/ shelter in place', 'End/relax stay at home/shelter in place', 'Closed non-essential businesses', 'Began to reopen businesses', 'Closed restaurants except take out', 'Reopen restaurants', 'Closed gyms', 'Reopened gyms', 'Closed movie theaters', 'Reopened movie theaters']]
rel_policies
# keep 'Initially reopen restaurants for outdoor dining only' for later -- non-datetime values

,State,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,Closed restaurants except take out,Reopen restaurants,Closed gyms,Reopened gyms,Closed movie theaters,Reopened movie theaters
0,Alabama,4/4/20,4/30/20,3/28/20,4/30/20,3/19/20,5/11/20,3/28/20,5/11/20,3/28/20,5/22/20
1,Alaska,3/28/20,4/24/20,3/28/20,4/24/20,3/18/20,4/24/20,3/18/20,5/8/20,3/18/20,5/8/20
2,Arizona,3/31/20,5/16/20,3/30/20,5/8/20,3/20/20,5/11/20,3/20/20,5/13/20,3/20/20,5/16/20
3,Arkansas,0,0,0,5/4/20,3/19/20,5/11/20,3/20/20,5/4/20,0,5/18/20
4,California,3/19/20,0,3/19/20,5/8/20,3/19/20,0,3/19/20,0,3/19/20,0
5,Colorado,3/26/20,4/27/20,3/26/20,5/1/20,3/17/20,5/27/20,3/17/20,6/2/20,3/17/20,0
6,Connecticut,0,0,3/23/20,5/20/20,3/16/20,5/20/20,3/16/20,6/17/20,3/16/20,6/17/20
7,Delaware,3/24/20,6/1/20,3/24/20,5/8/20,3/16/20,6/1/20,3/19/20,6/1/20,3/19/20,6/1/20
8,District of Columbia,4/1/20,5/29/20,3/25/20,5/29/20,3/16/20,5/29/20,3/17/20,0,3/17/20,0
9,Florida,4/3/20,5/18/20,0,5/18/20,3/20/20,5/18/20,3/20/20,5/18/20,3/20/20,0


In [12]:
testing = rel_policies.copy()

In [13]:
# changing 0 values to date (1900-01-01) and changing dates to the Saturday date of their weeks
def change_dates(df, col):
    df = df.replace('0', '19000101')
    df.loc[:, col] = pd.to_datetime(df.loc[:, col], errors='ignore')
    df[col] = df[col].where(df[col] == ((df[col] + Week(weekday=5)) - Week()), df[col] + Week(weekday=5))
    return df

In [15]:
rel_columns = ['Stay at home/ shelter in place', 'End/relax stay at home/shelter in place', 'Closed non-essential businesses', 'Began to reopen businesses', 'Closed restaurants except take out', 'Reopen restaurants', 'Closed gyms', 'Reopened gyms', 'Closed movie theaters', 'Reopened movie theaters']

for i in rel_columns:
    testing = change_dates(testing, i)
    
testing

,State,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,Closed restaurants except take out,Reopen restaurants,Closed gyms,Reopened gyms,Closed movie theaters,Reopened movie theaters
0,Alabama,2020-04-04,2020-05-02,2020-03-28,2020-05-02,2020-03-21,2020-05-16,2020-03-28,2020-05-16,2020-03-28,2020-05-23
1,Alaska,2020-03-28,2020-04-25,2020-03-28,2020-04-25,2020-03-21,2020-04-25,2020-03-21,2020-05-09,2020-03-21,2020-05-09
2,Arizona,2020-04-04,2020-05-16,2020-04-04,2020-05-09,2020-03-21,2020-05-16,2020-03-21,2020-05-16,2020-03-21,2020-05-16
3,Arkansas,1900-01-06,1900-01-06,1900-01-06,2020-05-09,2020-03-21,2020-05-16,2020-03-21,2020-05-09,1900-01-06,2020-05-23
4,California,2020-03-21,1900-01-06,2020-03-21,2020-05-09,2020-03-21,1900-01-06,2020-03-21,1900-01-06,2020-03-21,1900-01-06
5,Colorado,2020-03-28,2020-05-02,2020-03-28,2020-05-02,2020-03-21,2020-05-30,2020-03-21,2020-06-06,2020-03-21,1900-01-06
6,Connecticut,1900-01-06,1900-01-06,2020-03-28,2020-05-23,2020-03-21,2020-05-23,2020-03-21,2020-06-20,2020-03-21,2020-06-20
7,Delaware,2020-03-28,2020-06-06,2020-03-28,2020-05-09,2020-03-21,2020-06-06,2020-03-21,2020-06-06,2020-03-21,2020-06-06
8,District of Columbia,2020-04-04,2020-05-30,2020-03-28,2020-05-30,2020-03-21,2020-05-30,2020-03-21,1900-01-06,2020-03-21,1900-01-06
9,Florida,2020-04-04,2020-05-23,1900-01-06,2020-05-23,2020-03-21,2020-05-23,2020-03-21,2020-05-23,2020-03-21,1900-01-06


In [16]:
# changing dates to match Sophie's dataset
#http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timestamp-limitations

# testing['Stay at home/ shelter in place WEEK'] = testing['Stay at home/ shelter in place WEEK'].dt.strftime('%Y-%m-%d')
# mask = testing.iloc[:, 1].str.replace('-','').str[:8].astype(int).between(20200101, 20200501)
# print(mask)

In [17]:
# testing.iloc[:, 0] == testing["State"]

In [18]:
# replacing all prior-to-survey dates as first survey date (2020-05-02)
def fix_pre_survey_dates(df, col_number):
    df.iloc[:, col_number] = df.iloc[:, col_number].dt.strftime('%Y-%m-%d')
    mask = df.iloc[:, col_number].str.replace('-','').str[:8].astype(int).between(20200101, 20200501)
    df.iloc[:, col_number] = pd.to_datetime(df.iloc[:, col_number].mask(mask, '2020-05-02'))
    return df


In [19]:
# testing = fix_pre_survey_dates(testing, 1)
# testing

In [20]:
rel_col_nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in rel_col_nums:
    testing = fix_pre_survey_dates(testing, i)
    
testing

,State,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,Closed restaurants except take out,Reopen restaurants,Closed gyms,Reopened gyms,Closed movie theaters,Reopened movie theaters
0,Alabama,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-16,2020-05-02,2020-05-16,2020-05-02,2020-05-23
1,Alaska,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-09,2020-05-02,2020-05-09
2,Arizona,2020-05-02,2020-05-16,2020-05-02,2020-05-09,2020-05-02,2020-05-16,2020-05-02,2020-05-16,2020-05-02,2020-05-16
3,Arkansas,1900-01-06,1900-01-06,1900-01-06,2020-05-09,2020-05-02,2020-05-16,2020-05-02,2020-05-09,1900-01-06,2020-05-23
4,California,2020-05-02,1900-01-06,2020-05-02,2020-05-09,2020-05-02,1900-01-06,2020-05-02,1900-01-06,2020-05-02,1900-01-06
5,Colorado,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-02,2020-05-30,2020-05-02,2020-06-06,2020-05-02,1900-01-06
6,Connecticut,1900-01-06,1900-01-06,2020-05-02,2020-05-23,2020-05-02,2020-05-23,2020-05-02,2020-06-20,2020-05-02,2020-06-20
7,Delaware,2020-05-02,2020-06-06,2020-05-02,2020-05-09,2020-05-02,2020-06-06,2020-05-02,2020-06-06,2020-05-02,2020-06-06
8,District of Columbia,2020-05-02,2020-05-30,2020-05-02,2020-05-30,2020-05-02,2020-05-30,2020-05-02,1900-01-06,2020-05-02,1900-01-06
9,Florida,2020-05-02,2020-05-23,1900-01-06,2020-05-23,2020-05-02,2020-05-23,2020-05-02,2020-05-23,2020-05-02,1900-01-06


In [21]:
date0 = pd.Timestamp('1900-01-06')
date1 = pd.Timestamp('2020-05-02')
date2 = pd.Timestamp('2020-05-09')
date3 = pd.Timestamp('2020-05-16')
date4 = pd.Timestamp('2020-05-23')
date5 = pd.Timestamp('2020-05-30')
date6 = pd.Timestamp('2020-06-06')
date7 = pd.Timestamp('2020-06-13')

dates = [date0, date1, date2, date3, date4, date5, date6, date7]
df1 = pd.DataFrame({'Dates':dates})


In [22]:
# copy previous table
policy_weeks = testing.copy()

# create 6 tables - one for each week
policy_weeks1 = policy_weeks.copy()
policy_weeks1['w1'] = date1

policy_weeks2 = policy_weeks.copy()
policy_weeks2['w1'] = date2

policy_weeks3 = policy_weeks.copy()
policy_weeks3['w1'] = date3

policy_weeks4 = policy_weeks.copy()
policy_weeks4['w1'] = date4

policy_weeks5 = policy_weeks.copy()
policy_weeks5['w1'] = date5

policy_weeks6 = policy_weeks.copy()
policy_weeks6['w1'] = date6

In [23]:
# testing['w1'] = date1

In [24]:
# testing = testing.drop('w1', axis=1)
# testing

In [25]:
def count_policy1(col):
    p = 0
    for i in col:
        if i == date1:
            p = 1
        else:
            p = 0
    return p

def count_policy2(col):
    p = 0
    for i in col:
        if i == date2:
            p = 1
        else:
            p = 0
    return p

def count_policy3(col):
    p = 0
    for i in col:
        if i == date3:
            p = 1
        else:
            p = 0
    return p

def count_policy4(col):
    p = 0
    for i in col:
        if i == date4:
            p = 1
        else:
            p = 0
    return p

def count_policy5(col):
    p = 0
    for i in col:
        if i == date5:
            p = 1
        else:
            p = 0
    return p

def count_policy6(col):
    p = 0
    for i in col:
        if i == date6:
            p = 1
        else:
            p = 0
    return p

In [26]:
week1 = policy_weeks1.groupby('State').agg(count_policy1)
week1['w1'] = date1
# week1
week2 = policy_weeks2.groupby('State').agg(count_policy2)
week2['w1'] = date2

week3 = policy_weeks3.groupby('State').agg(count_policy3)
week3['w1'] = date3

week4 = policy_weeks4.groupby('State').agg(count_policy4)
week4['w1'] = date4

week5 = policy_weeks5.groupby('State').agg(count_policy5)
week5['w1'] = date5

week6 = policy_weeks6.groupby('State').agg(count_policy6)
week6['w1'] = date6

In [27]:
# week6

In [28]:
# testing

In [29]:
policies_6weeks = pd.concat([week1, week2, week3, week4, week5, week6])
# policies_6weeks

In [30]:
policies_6weeks = policies_6weeks.rename(columns={"w1": "Dates"})
# policies_6weeks = policies_6weeks.set_index(['Dates'])

policies_6weeks = policies_6weeks.reset_index().set_index(['State', 'Dates'])

In [31]:
policies_6weeks["Total # Policies"] = policies_6weeks.sum(axis=1)
policies_6weeks

,,Stay at home/ shelter in place,End/relax stay at home/shelter in place,Closed non-essential businesses,Began to reopen businesses,Closed restaurants except take out,Reopen restaurants,Closed gyms,Reopened gyms,Closed movie theaters,Reopened movie theaters,Total # Policies
State,Dates,,,,,,,,,,,
Alabama,2020-05-02,1,1,1,1,1,0,1,0,1,0,7
Alaska,2020-05-02,1,1,1,1,1,1,1,0,1,0,8
Arizona,2020-05-02,1,0,1,0,1,0,1,0,1,0,5
Arkansas,2020-05-02,0,0,0,0,1,0,1,0,0,0,2
California,2020-05-02,1,0,1,0,1,0,1,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
Virginia,2020-06-06,0,0,0,0,0,0,0,0,0,0,0
Washington,2020-06-06,0,1,0,0,0,0,0,0,0,0,1
West Virginia,2020-06-06,0,0,0,0,0,0,0,0,0,1,1


In [43]:
policies_6weeks.to_csv('policies.csv')

In [ ]:
# EDA & VISUALIZATIONS

In [37]:
total_policies_week = policies_6weeks.groupby('Dates').sum()
# total_policies_week

In [39]:
total_policies_state = policies_6weeks.groupby('State').sum()
# total_policies_state